In [1]:
import datetime

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from scipy.optimize import minimize

Dataset preparation

In [11]:
#Import of four market indices: US total stock index (VTI), US aggregate bond index (AGG), US commodity index (DBC) and Volatility Index (VIX)
AGG_DB = pd.read_csv("DataBase/AGG.csv", delimiter=",")
DBC_DB = pd.read_csv("DataBase/DBC.csv", delimiter=",")
VIX_DB = pd.read_csv("DataBase/VIX.csv", delimiter=",")
VTI_DB = pd.read_csv("DataBase/VTI.csv", delimiter=",")


In [6]:
AGG_DB.head()




,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-01-03,61.855000,62.730000,61.430000,62.575001,44.687496,1769800
1,2006-01-04,62.660000,63.029999,62.625000,63.029999,45.012447,763600
2,2006-01-05,62.985001,63.055000,62.779999,62.990002,44.983898,258200
3,2006-01-06,63.415001,63.634998,63.115002,63.610001,45.426636,436000
4,2006-01-09,63.665001,63.939999,63.610001,63.845001,45.594460,689600


In [7]:
DBC_DB.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-02-07,24.049999,24.200001,23.450001,23.500000,20.285254,674400
1,2006-02-08,23.490000,23.670000,23.330000,23.400000,20.198936,335400
2,2006-02-09,23.559999,23.820000,23.510000,23.620001,20.388842,705900
3,2006-02-10,23.650000,23.690001,23.100000,23.190001,20.017660,276700
4,2006-02-13,23.190001,23.200001,22.770000,22.830000,19.706907,237600


In [8]:
VIX_DB.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-01-23,20.209999,20.330000,19.549999,19.809999,19.809999,0.0
1,2023-01-24,19.889999,20.469999,18.910000,19.200001,19.200001,0.0
2,2023-01-25,19.559999,20.900000,18.990000,19.080000,19.080000,0.0
3,2023-01-26,19.049999,19.480000,18.670000,18.730000,18.730000,0.0
4,2023-01-27,18.900000,19.000000,17.969999,18.510000,18.510000,0.0


In [9]:
VTI_DB.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2006-01-03,61.855000,62.730000,61.430000,62.575001,44.687496,1769800
1,2006-01-04,62.660000,63.029999,62.625000,63.029999,45.012447,763600
2,2006-01-05,62.985001,63.055000,62.779999,62.990002,44.983898,258200
3,2006-01-06,63.415001,63.634998,63.115002,63.610001,45.426636,436000
4,2006-01-09,63.665001,63.939999,63.610001,63.845001,45.594460,689600
